In [ ]:
from pathlib import Path

import numpy as np
import xarray as xr
from datatree import DataTree, open_datatree
from xarray import open_zarr

## Data Creation


In [ ]:
xda = xr.DataArray(
    np.arange(3 * 18).reshape(3, 18),
    coords={"label": list("abc"), "z": list(range(18))},
)
xda = xda.chunk({"label": 2, "z": 4})
xda

In [ ]:
xdt = DataTree(xr.Dataset({"my_xda": xda}))
xdt.my_xda

## Data Writing


In [ ]:
zarr_path = Path() / "../generated/my_array.zarr"
xdt.to_zarr(zarr_path)

## Data Reading


### Using xarray's `open_zarr` 

See https://docs.xarray.dev/en/stable/generated/xarray.open_zarr.html

Documentation version: `stable` at time of writing: 11 nov 2023

Resulting behaviours serve as reference.

#### No `chunks` kwarg

Stored chunks are used.

In [ ]:
open_zarr(zarr_path).my_xda

#### With `chunks='auto'`

Stored chunks are used.

Same behaviour as with no `chunks` kwarg.

In [ ]:
# https://docs.xarray.dev/en/stable/generated/xarray.open_zarr.html
open_zarr(zarr_path, chunks="auto").my_xda

### Using datatree's `open_datatree` with `engine='zarr'`

See https://xarray-datatree.readthedocs.io/en/latest/generated/datatree.open_datatree.html#datatree.open_datatree

Documentation version:

```
Datatree 0.0.14.dev5+g433f78d.d20231110 documentation
```

#### No `chunks` kwarg

No chunking performed.

(NOK)

(!) Differs from the xarray's reference behaviour where stored chunks are used.

In [ ]:
open_datatree(zarr_path, engine="zarr").my_xda

#### With `chunks='auto'`

A chunk identical to the shape of the data is used.
This means chunking is useless as there is only a single chunk representing the whole dataset

(NOK)

(!) Differs from the xarray's reference behaviour where stored chunks are used.


In [ ]:
open_datatree(zarr_path, engine="zarr", chunks="auto").my_xda

#### With `chunks` kwarg (same as stored chunks)

(OK)

No warning is shown because given chunks correspond to the stored chunks

In [ ]:
open_datatree(zarr_path, engine="zarr", chunks={"label": 2, "z": 4}).my_xda

#### With `chunks` kwarg (differing from the stored chunks)

(OK)

A warning is shown

> UserWarning: The specified chunks separate the stored chunks along dimension "z" starting at index 5. This could degrade performance. Instead, consider rechunking after loading.

In [ ]:
open_datatree(zarr_path, engine="zarr", chunks={"label": 999, "z": 5}).my_xda